In [9]:
import sys

import pandas as pd
## common PDB access
pdsDir = globals()['_dh'][0]
sys.path.insert(1, pdsDir+'/..')
import PDBinfrastructure as infra
import myDetails

In [10]:
#####################
### access PDB
#####################
deets=myDetails.SetITk()
myClient=infra.AuthenticateUser(deets.ac1,deets.ac2)
user=myClient.get('getUser', json={'userIdentity': myClient.user.identity})
user['firstName']
myClient.user.expires_in

1798

In [11]:
#####################
### checking PDB
#####################

### projects
projList = list(myClient.get('listProjects'))
projList
df_proj= pd.DataFrame([{"name":x['name'],"code":x['code']} for x in projList])
df_proj

### sub-projects
myProjCode="P"
subprojList = myClient.get('listSubprojects', json={'project':myProjCode})
subprojList
df_subproj= pd.DataFrame([{"name":x['name'],"code":x['code']} for x in subprojList])
df_subproj

### institutions
instList = list(myClient.get('listInstitutions'))
instList
df_inst= pd.DataFrame([{"name":x['name'],"code":x['code']} for x in instList])
df_inst.query('name.str.contains("G")')

### component types
compTypesList = list(myClient.get('listComponentTypes', json={'project':myProjCode}))
compTypesList
df_compType= pd.DataFrame([{"name":x['name'],"code":x['code'],"id":x['id']} for x in compTypesList])
df_compType

,name,code,id
0,Bare Module,BARE_MODULE,5f1d2df071645a000b7b2c85
1,Benchmark,BENCHMARK,5d715cc4b21e82000b0913e3
2,Coupled Ring,COUPLED_RING,602431aa86dec3000ad01dba
3,Coupled Ring to PP0 Flex,COUPLED_RING_TO_PP0_FLEX,601812d45f2a42000a8d1ae8
4,Data Flex,DATA_FLEX,602431a586dec3000ad01da6
...,...,...,...
117,test chip monika,TESTCHIPMONIKA,5f070fd0e3f2d1000a87261e
118,test glue,test_glue,601adf745f2a42000a8d6ac9
119,test material,test_material,601ab4bf5f2a42000a8d6664
120,test module,TESTMODULE,5e9452775fbf4f000a5c24a1


In [ ]:
#####################
### deletion
#####################

# get my components (ready + requestedToDelete + deleted)
myComps= myClient.get('listMyComponents')
# myComps.page_size
# myComps.page_info
# myComps.page_index

df_myComps= pd.DataFrame([{"type":x['componentType']['code'],"state":x['state'],"code":x['code']} for x in myComps.data])
df_myComps.describe()
df_myComps.pivot_table(index=['type','state'], aggfunc='size')

# subset if required
theseComps=[x for x in myComps.data if x['componentType']['code']=="SENSOR_TILE"]
len(theseComps)

# loop to approve delete requests
for mc in theseComps:
    if mc['state']=="requestedToDelete":
        try:
        #print(mc['code'])
            print(myClient.post('approveDeleteComponent',json={'component':mc['code'], 'approve':"true"}))
        except:
            print("issue with:",mc['code'])